### Linear-probe evaluation

In [1]:
import os 
import clip
import torch 
import numpy as np 
from sklearn.linear_model import LogisticRegression 
from torch.utils.data import DataLoader 
from torchvision.datasets import CIFAR100 
from tqdm import tqdm 

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)

In [4]:
root = os.path.expanduser("~/.cache")
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for image, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(image.to(device))
            all_features.append(features)
            all_labels.append(labels)
    
    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [6]:
train_feature, train_labels = get_features(train)
test_feature, test_labels = get_features(test)

100%|██████████| 100/100 [00:09<00:00, 10.10it/s]


In [8]:
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_feature, train_labels)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        51300     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.60517D+00    |proj g|=  1.53230D-02


 This problem is unconstrained.



At iterate   50    f=  6.71765D-01    |proj g|=  1.09619D-02

At iterate  100    f=  5.82953D-01    |proj g|=  3.75853D-03

At iterate  150    f=  5.66864D-01    |proj g|=  1.82540D-03

At iterate  200    f=  5.63202D-01    |proj g|=  1.09593D-03

At iterate  250    f=  5.62574D-01    |proj g|=  7.81246D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
51300    295    304      1     0     0   8.157D-05   5.624D-01
  F =  0.56242531778965943     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


LogisticRegression(C=0.316, max_iter=1000, random_state=0, verbose=1)

In [10]:
predictions = classifier.predict(test_feature)
accuracy = np.mean(test_labels == predictions).astype(float) * 100. 
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 80.070
